[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PaiPai121/stable-diffusion-webui-colab-googledrive/blob/main/sd_webui_colab_googledrive.ipynb)

In [ ]:
INPUTFILENAME = "Sophon"

#切换到python3.10
!wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
!bash ./py310.sh -b -f -p /usr/local
!python -m ipykernel install --name "py310" --user

#查看是什么GPU
!nvidia-smi
#挂载谷歌硬盘
from google.colab import drive
drive.mount('/content/drive/')
!echo "google硬盘挂载完成."

# 安装aria2下载器，便于下载模型等
!apt -qq install liblz4-tool aria2

#从Akegarasu的git仓库下载Lora训练模型
!git clone --recurse-submodules https://github.com/Akegarasu/lora-scripts
!echo "git的lora库克隆完成."
!cd lora-scripts && git pull && git submodule update --init --recursive
!echo "更新完成."

#安装其他依赖
%cd /content/lora-scripts/sd-scripts/
!pip -q install --upgrade -r requirements.txt

#安装xformers 0.0.16版本
!pip install xformers==0.0.16

#安装Triton
!pip -q install triton==2.0.0.dev20221202

#安装lion
!pip -q install --upgrade lion-pytorch locon

#从谷歌硬盘中拷贝你之前上传的训练材料
!mkdir -p /content/lora-scripts/train/$INPUTFILENAME
!cp -r /content/drive/MyDrive/Lora/input/* /content/lora-scripts/train/$INPUTFILENAME
!echo "copy训练材料完成."

#拷贝正则化图片
!mkdir -p /content/lora-scripts/train/reg/
!cp -r /content/drive/MyDrive/Lora/reg/* /content/lora-scripts/train/reg/
!echo "copy正则化图片完成."

%cd /content/lora-scripts

import os
def downloadModels(url):
    os.system('aria2c -s 5 -x 10 -d /content/lora-scripts/sd-models '+ url)
    os.system('echo "下载完成"')

######################################## 下载模型部分，可修改

# 默认先下载这几个model，如果有其他想要的可以继续加
urls = ['https://civitai.com/api/download/models/11745', # chilloutmix
        'https://civitai.com/api/download/models/16619', # grapefruithentaimodel
        'https://civitai.com/api/download/models/7425', # counterfeit
        ]

for url in urls:
    # 下载list中的所有models
    downloadModels(url)

def moveModelsfromGoogleDrive():
    path = "/content/drive/MyDrive/stable-diffusion-webui/models/Stable-diffusion/*" # 一个示例，MyDrive后面是模型在google drive中的路径
    os.system("cp -r "+ path + " /content/lora-scripts/sd-models")

# moveModelsfromGoogleDrive() # 需要使用时打开



########################################
########################################
# 调整训练参数等
# 此处我在云盘中保存了一个调整好的训练文件 train.sh并放在了谷歌云盘下的lora-scrpits/train.sh
########################################
########################################

!cp -r /content/drive/MyDrive/lora/train.sh /content/lora-scripts

如有需要继续修改，打开*/content/lora-scripts/train.sh*


In [ ]:
##################注意！！！请先修改配置文件！！！###########################

## 每次修改完训练文件后运行这个cell即可进行训练，如果想再执行从谷歌云盘中将图片素材拷贝入colab请先执行下一个cell

#开始训练！
%cd /content/lora-scripts/
!bash train.sh

!echo "完成了 XXXD."
!cp /content/lora-scripts/output/ /content/drive/MyDrive/lora/output

In [ ]:
#从谷歌硬盘中拷贝你之前上传的训练材料
!mkdir -p /content/lora-scripts/train/Sophon/
!cp -r /content/drive/MyDrive/Lora/input/* /content/lora-scripts/train/Sophon/
!echo "copy训练材料完成."

#拷贝正则化图片
!mkdir -p /content/lora-scripts/train/reg/
!cp -r /content/drive/MyDrive/Lora/reg/* /content/lora-scripts/train/reg/
!echo "copy正则化图片完成."
#接下来请运行训练等cell